In [18]:
# prerequisite: 
# run 
#      ./printEnergyCorrections -s ../../setup/setup.json -t ../../setup/target.json -d U4,U5 -E {insert peaks one wishes to fit after} -i {implantation depth of given isotope} > {filename}.dat
# with data output to
#      "correctedEnergiesU4U5.dat"

In [10]:
import numpy as np
import uproot
import matplotlib.pyplot as plt

In [11]:
#load in generated file from above
Es = np.loadtxt('/home/haue/repositories/Masters/calibration/dssd_calibration/U5_energyestimation.dat').flatten()
print(len(Es)/4)

49.0


In [12]:
from scipy.optimize import curve_fit
def lin(x, a, b):
    return a*x + b

In [13]:
"""
([(385.72237105453763, 0.5786771607068808),
  (1843.1815428519299, 3.8391338032060935),
  (2217.4549852704345, 1.9202145004587217),
  (4651.192036484095, 4.798722937645029),
  (5402.617947801247, 1.9202145004587354)],
 [(904.0191131421628, 1.920220495723927),
  (2076.7425660534773, 3.8391368018535106),
  (3337.756164355824, 4.798725336660437),
  (4089.1820756729758, 1.920220495723936)],
 (1730.8841553184843, 3.8391483166378255),
"""
nums =  ["5"]
sides = ["F", "B"]
Es0 = np.array([904.02,1843.18, 2076.74, 2217.45, 3337.75,4089.18,5402.61])
Es0_errs = np.array([1.92,3.84, 3.84, 1.92, 4.79, 1.92, 1.92])

j = 0
for num in nums:
        for side in sides:
            for strip in range(2, 15+1, 1):
                
                print([Es[7*j+i] for i in range(len(Es0))])
                j += 1

[895.068, 1837.95, 2071.55, 2212.74, 3334.18, 4086.21, 5400.02]
[892.176, 1835.93, 2069.85, 2211.15, 3333.09, 4085.39, 5399.4]
[887.968, 1834.48, 2068.28, 2209.18, 3331.64, 4084.17, 5398.61]
[886.478, 1832.41, 2067.08, 2208.4, 3330.78, 4083.16, 5397.56]
[883.727, 1831.49, 2065.7, 2206.85, 3329.99, 4082.36, 5397.07]
[882.614, 1829.81, 2064.24, 2205.45, 3328.6, 4081.74, 5396.61]
[879.346, 1828.17, 2063.31, 2204.82, 3328.08, 4081.08, 5395.93]
[879.619, 1826.99, 2062.33, 2203.27, 3327.43, 4080.48, 5395.53]
[876.666, 1826.75, 2062.34, 2202.85, 3326.97, 4080.14, 5395.11]
[876.098, 1825.82, 2060.86, 2202.08, 3326.34, 4079.81, 5394.84]
[874.666, 1826.32, 2060.77, 2202.39, 3326.43, 4079.49, 5394.49]
[874.943, 1825.52, 2060.23, 2201.62, 3326.01, 4079.32, 5394.58]
[875.006, 1825.82, 2060.73, 2201.91, 3326.17, 4079.07, 5394.73]
[875.307, 1826.28, 2060.87, 2202.58, 3326.66, 4079.72, 5394.95]
[876.545, 1826.55, 2060.94, 2202.36, 3326.38, 4079.97, 5394.95]
[875.609, 1825.29, 2060.38, 2202.02, 3326.72

In [14]:
"""
dead_strips = ["U5F2"]
def count_in_range(xs, low, high):
    return np.sum((xs >= low) & (xs <= high))
nums =  ["5"]
sides = ["F", "B"]
with uproot.open("/home/haue/repositories/Masters/data/cal/pha.root") as file:
    for num in nums:
        for side in sides:
            if side == "F":
                print("# Front")
            else:
                print("# Back")
            for strip in range(1, 16+1, 1):
                if strip == 1 or strip == 16:
                    print(0, 0)
                    continue
                base = "U" + num + side + str(strip)
                #skip dead strips
                if base in dead_strips:
                    print(0, 0)
                    j += 1
                    continue
                y1, x1 = file[base].to_numpy()
                x0 = np.min(x1) ## starting bin, in this case just 90
                x1 = x1[:-1] + 0.5
                # tspectrum
                y2, x2 = file[base + "A"].to_numpy()
                x2 = x2[:-1] + 0.5 + x0
                # peak positions, in found order
                x3, y3 = file[base + "P"].values()
                x3 = x3 + x0
                x3_sortedx = x3[np.argsort(x3)] # sorted by low to high x
                y3_sortedx = y3[np.argsort(x3)]
                print("x3 and y3 values", list(zip(x3,y3)))

                while len(x3_sortedx) > len(Es0) + 1:
                    # Count how many peaks still remain in protected regions
                    count_230_280 = count_in_range(x3_sortedx, 230, 280)
                    count_500_650 = count_in_range(x3_sortedx, 500, 650)

                    # Build mask of deletable peaks (i.e., not in protected zones)
                    deletable = np.ones(len(x3_sortedx), dtype=bool)
                    for i, xval in enumerate(x3_sortedx):
                        if 230 <= xval <= 280 and count_230_280 <= 1:
                            deletable[i] = False
                        if 500 <= xval <= 650 and count_500_650 <= 2:
                            deletable[i] = False

                    if not np.any(deletable):
                        break  # nothing deletable left, exit

                    # Choose lowest y among deletable peaks
                    masked_y = np.where(deletable, y3_sortedx, np.inf)
                    idx = np.argmin(masked_y)

                    # Update counters if removing a protected peak
                    if 230 <= x3_sortedx[idx] <= 280:
                        count_230_280 -= 1
                    if 500 <= x3_sortedx[idx] <= 650:
                        count_500_650 -= 1

                    # Remove peak
                    x3_sortedx = np.delete(x3_sortedx, idx)
                    y3_sortedx = np.delete(y3_sortedx, idx)
                    
                    
                plt.figure()
                plt.plot(x2,y2)
                for x,y in zip(x3_sortedx,y3_sortedx):
                    plt.plot(x, y, 'rs')
                plt.xlim(0, 2000)
                plt.title(base)
                plt.show()
                plt.close()
"""

'\ndead_strips = ["U5F2"]\ndef count_in_range(xs, low, high):\n    return np.sum((xs >= low) & (xs <= high))\nnums =  ["5"]\nsides = ["F", "B"]\nwith uproot.open("/home/haue/repositories/Masters/data/cal/pha.root") as file:\n    for num in nums:\n        for side in sides:\n            if side == "F":\n                print("# Front")\n            else:\n                print("# Back")\n            for strip in range(1, 16+1, 1):\n                if strip == 1 or strip == 16:\n                    print(0, 0)\n                    continue\n                base = "U" + num + side + str(strip)\n                #skip dead strips\n                if base in dead_strips:\n                    print(0, 0)\n                    j += 1\n                    continue\n                y1, x1 = file[base].to_numpy()\n                x0 = np.min(x1) ## starting bin, in this case just 90\n                x1 = x1[:-1] + 0.5\n                # tspectrum\n                y2, x2 = file[base + "A"].to_numpy

In [ ]:
dead_strips = ["U5F2"]
def count_in_range(xs, low, high):
    return np.sum((xs >= low) & (xs <= high))
nums =  ["5"]
sides = ["F", "B"]
j = 0
res0 = res = np.array([])
with uproot.open("/home/haue/repositories/Masters/data/cal/pha.root") as file:
    for num in nums:
        for side in sides:
            if side == "F":
                print("# Front")
            else:
                print("# Back")
            for strip in range(1, 16+1, 1):
                if strip == 1 or strip == 16:
                    print(0, 0)
                    continue
                base = "U" + num + side + str(strip)
                #skip dead strips
                if base in dead_strips:
                    print(0, 0)
                    j += 1
                    continue
                # data
                y1, x1 = file[base].to_numpy()
                x0 = np.min(x1) ## starting bin, in this case just 90
                x1 = x1[:-1] + 0.5
                # tspectrum
                y2, x2 = file[base + "A"].to_numpy()
                x2 = x2[:-1] + 0.5 + x0
                # peak positions, in found order
                x3, y3 = file[base + "P"].values()
                x3 = x3 + x0
                # fitted peak positions and their errors
                fx, fx_err = file[base + "F"].values()

                
                # peak positions, sorted xmin to xmax
                x3_sortedx = x3[np.argsort(x3)] # sorted by low to high x
                y3_sortedx = y3[np.argsort(x3)]
                fx = fx[np.argsort(fx)]
                fx_err = fx_err[np.argsort(fx)]
                
                if len(x3_sortedx)!=len(fx):
                    raise ValueError(f"Length mismatch: len(x3_sortedx)={len(x3_sortedx)} is not equal to len(fx)={len(fx)}, somehow the fitting procedure messed up..")



                # proton peaks pV, pVI, ..., pVIII
                ##Hand held peak pruning
                while len(x3_sortedx) > len(Es0):
                    # Count how many peaks still remain in protected regions
                    count_230_280 = count_in_range(x3_sortedx, 230, 280)
                    count_500_650 = count_in_range(x3_sortedx, 500, 650)

                    deletable = np.ones(len(x3_sortedx), dtype=bool)
                    for i, xval in enumerate(x3_sortedx):
                        if 230 <= xval <= 280 and count_230_280 <= 1:
                            deletable[i] = False
                        if 500 <= xval <= 650 and count_500_650 <= 2:
                            deletable[i] = False

                    if not np.any(deletable):
                        break  # nothing deletable left
                    
                    masked_y = np.where(deletable, y3_sortedx, np.inf)
                    idx = np.argmin(masked_y)

                    # Update counters if removing a protected peak
                    if 230 <= x3_sortedx[idx] <= 280:
                        count_230_280 -= 1
                    if 500 <= x3_sortedx[idx] <= 650:
                        count_500_650 -= 1
                        
                    x3_sortedx = np.delete(x3_sortedx, idx)
                    y3_sortedx = np.delete(y3_sortedx, idx)
                

                
                ### the gaussian fitting routine is very simple and therefore one should expect errors
                ### This would explain why x3_sortedx is the better parameter to use for now
                #chs = fx
                chs = x3_sortedx
                popt0, pcov0 = curve_fit(lin, chs, Es0, sigma=Es0_errs, p0=[2, 0])
                Ecs = np.array([Es[7*j+i] for i in range(len(x3_sortedx))])
                popt, pcov = curve_fit(lin, chs, Ecs, sigma=Es0_errs, p0=[2, 0])
                
                res0 = np.append(res0, Es0 - lin(chs, *popt0)) ## not energy corrected peak energies
                res  = np.append(res, Ecs - lin(chs, *popt)) ## energy corrected peak energies ! the most important one!
                
                
                ########################################################################################################################
                ## Plotting routine for chosen peaks
                """
                plt.figure()
                plt.plot(x2,y2)
                for x,y in zip(x3_sortedx,y3_sortedx):
                    plt.plot(x, y, 'rs')
                plt.xlim(0, 2000)
                plt.title(base)
                plt.show()
                plt.close()
                """              
                ######################################################################################################################
                ## plotting routine for quality control
                #"""
                fig, axes = plt.subplots(2, 1,gridspec_kw={'height_ratios': [4, 1], 'hspace': 0}, sharex=True)
                ax1, ax2 = axes
                #plt.sca(axes[0]) ## sets current axes
                ax1.errorbar(chs, Ecs, yerr=Es0_errs, capsize=2, marker='o',markersize=3.5, ls='', color='r', label="Data points")
                ax1.plot([50, 2000], lin(np.array([50, 2000]), *popt), label=f"fit vals: {np.round(popt[1],6)} {np.round(popt[0],6)}", color='k', ls='--')
                ax1.legend()
                ax1.set_ylabel("Energy(keV)")
                ax1.set_title(f"Linear fit to {base}")
                #plt.sca(axes[1])
                ax2.scatter(chs, Ecs - lin(chs, *popt), color='k', s=8)
                ax2.hlines(0,50,2000, ls='--', color='k', lw=0.7)
                ax2.set_yticks(np.arange(-10,10, 2.5), minor=True)
                ax2.set_ylim(-np.max(np.abs(Ecs - lin(chs, *popt)))-2, np.max(np.abs(Ecs - lin(chs, *popt)))+2)
                
                ax2.set_ylabel("Residuals (keV)")
                ax2.set_xlabel("Channel")
                
                plt.show()
                
                
                #"""
                #### output
                print(popt[1], popt[0])
                j += 1


# Front
0 0
0 0
-20.355985557518448 3.892041537350297
-28.338958955511426 3.7261326841329163
-29.94341440491912 3.6359381339259613
-39.877601310944264 3.6696574075064765
-23.137591987687554 3.6733238245630693
-39.559209606709345 3.620444116053983
-31.35209838697284 3.6929535269583775
-15.575343232627127 4.000616614315484
-29.700304759058817 3.632302505518476
-11.65914453548383 3.6611127980271285
-24.84943113780476 3.6623318435161147
-20.719041854238245 3.959395965591707
-32.50208019953964 3.795944374607236
0 0
# Back
0 0
-35.9252611891646 3.7618584126621597
-11.174364613603633 3.6740253246719625
-27.1157693640427 3.7635357619533685
-7.793206194580674 3.799619732578849
-25.32762824176032 3.6724718327557055
-3.075801627050911 3.602164330884189
-16.984727350854723 4.059270485742002
-43.36093873584435 3.8099896807214346
-23.142462750802622 3.88843604659182
-41.947011049236565 3.9270269509744122
-25.90393577252043 3.7423542416923796
-37.094539100612266 3.715617270826934
-17.637894877777278 

In [ ]:
np.savetxt("res_proton_uncorrected.dat", res0)
np.savetxt("res_proton_corrected.dat", res)